In the output values, how many times do digits 1, 4, 7, or 8 appear?

In [98]:


final = 1

if not final:
    data = open(r"data0.txt").readlines()
else:
    data = open(r"data1.txt").readlines()

input = []
output = []

for row in data:
    input_item, output_item = row.split('|')
    input.append(input_item.strip())
    output.append(output_item.strip())

count_digits = 0
for o in output:
    digits = o.split(' ')
    count_digits += sum(map(lambda s: len(s) in [2, 3, 4, 7], digits))

print(count_digits)

355


For each entry, determine all of the wire/segment connections and decode the four-digit output values. What do you get if you add up all of the output values?

In [115]:
segments = {key: None for key in 'ABCDEFG'}

grand_total = 0

# sort the result of the lookups in s
def join_sorted(s: dict, elements: str) -> str:
    string = ''
    for e in elements:
        string += s[e]
    return ''.join(sorted(string))

# for each input row
for i, inp in enumerate(input):

    # sort the input digits into length buckets
    lengths = {}
    for c in range(2,8):
            lengths[c] = []
    digits:list = inp.split(' ')
    for digit in digits:
        digit = sorted(digit)
        lengths[len(digit)].append(digit)


    # lengths 2, 3, and 4 will have just one example
    s2 = set(lengths[2][0])
    s3 = set(lengths[3][0])
    s4 = set(lengths[4][0])

    # set theory to the rescue
    # common = appears in all elements (intersections)
    # unique = apears in only one element (union of subtractions)
    # twos = appears in only two elements (union minus common minus unique)
    s5common = set(lengths[5][0]) & set(lengths[5][1]) & set(lengths[5][2])
    s5unique = (set(lengths[5][0]) - set(lengths[5][1]) - set(lengths[5][2])) | (set(lengths[5][1]) - set(lengths[5][0]) - set(lengths[5][2])) | (set(lengths[5][2]) - set(lengths[5][1]) - set(lengths[5][0])) 
    s6common = set(lengths[6][0]) & set(lengths[6][1]) & set(lengths[6][2])
    s6twos = (set(lengths[6][0]) | set(lengths[6][1]) | set(lengths[6][2])) - s6common # there are no s6uniques

    # logic the crap out of it
    # results are stored as sets with a single element
    # keep them as sets to make the math easier

    # A is the top of the seven
    segments["A"] = s3 - s2

    # B is 4 minus 1 minus the common elements of 2, 3, and 5
    segments["B"] = s4 - s2 - s5common

    # D is 4 minus 1 minus B
    segments["D"] = s4 - s2 - segments["B"]

    # E is the unique elements of 2, 3, and 5 minus B
    segments["E"] = s5unique - segments["B"]

    # G is the intersecton of the common elements of (2, 3, 5) and the common elements of (0, 6, 9), minus A
    segments["G"] = (s5common & s6common) - segments["A"]

    # F is the common elements of (0, 6, 9), minus A, B, and G
    segments["F"] = s6common - segments["A"] - segments["B"] - segments["G"]

    # C is the twos of (0, 6, 9) minus D and E
    segments["C"] = s6twos - segments["D"] - segments["E"]

    # pop out of the sets to create a dictionary with just strings  
    s = {key:value.pop() for key, value in segments.items()}



    # build the sorted digit patterns into a dictionary
    digit_patterns = {        
        join_sorted(s, 'ABCEFG'): '0',
        join_sorted(s, 'CF') : '1',
        join_sorted(s, 'ACDEG'): '2',
        join_sorted(s, 'ACDFG'): '3',
        join_sorted(s, 'BCDF') : '4',
        join_sorted(s, 'ABDFG'): '5',
        join_sorted(s, 'ABDEFG'): '6',
        join_sorted(s, 'ACF'): '7',
        join_sorted(s, 'ABCDEFG'): '8',
        join_sorted(s, 'ABCDFG'): '9'
    }

    # sort the output strings and look them up
    output_number = []
    for o in output[i].split(' '):
        output_number.append(digit_patterns[''.join(sorted(o))])
    
    # make it an int and add to the grand total
    grand_total += int(''.join(output_number))


print(f"The output number total is {grand_total}")



The output number total is 983030
